### Objective

###### Situation
Excel, OBIEE , Tableau & Power BI are few of many reporting tools that currently exist in market and are used for presenting Weekly Business Report (WBR) to leadership . Each tool comes with its own pros and cons. Excel is probably the most used reporting tool  becasue of two major reason easy learning curve with multiple functions and also accesiable without any limitations, howver pros include laginess due to multiple formulas getting refreshed also with increase in data size dashboard faces multiple issues with refresh and loding . On the other side BIE tools such as Tableau also takes the advantage of easy learning curve but comes with high price for licensing and also dev support for server maintainance.

###### Proposed solution

Using pandas and easy-to-use data structures and data analysis tools for the Python programming language to replicate the same view that is created using Tableau or excel . Why Pandas ? Pandas comes with multiple powerful functionalities that let any user to perform tasks on data set that can be done using SQL, Excel and tableau combined. Being created on top of Python pandas let user write loops , functions , classes and other features of a programming langauge making complete end to end operation controlled by user without any feature limitations. Using pandas will also avoid dashboard update lags due to increase in size of the data.

###### Impact

* Cost effective - Pandas is opensource and requires no additional licensing cost.
* Compatible with other packages - Pandas can be combined with other packages such as bokeh , matplotlib even with django to create an interactive view. 
* Easy learning curve - Not in comparission to excel or tableau but pandas when compared to other langauges can bank on easy learning curve.
* Any IO format - Pandas can read data directly from SQL, excel , pdf and many other sources. Final output can also be shared in any required format csv , excel , html and more.

###### Final Output

By the end of the complete process a business report showing output for any dimensions at week , month , quarter and ytd grain(and can be controlled easily for any time range) from a raw sales data taken from Superstore sales.


### import packages required


In [431]:
import pandas as pd
from functools import reduce

### Load data as csv

In [375]:
df = pd.read_csv('Sample - Superstore.csv')

### Explore Data Content

In [376]:
df.shape #to check total number of rows and columns. Format:(rows,columns)

(9994, 21)

In [377]:
df.dtypes #to check data types for each column

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code      float64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object

In [378]:
df.isnull().sum()  #to check number of missing values for each column

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
dtype: int64

We only have missing values for postal code, we can avoid dealing with it for our use case.

In [432]:
df.head() # to check sample data set

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Order Year,Order quarter,Order Month,year_week,year_month,year_quarter,Order Week Name,Order Month Name,Order quarter Name,Order year Name
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,2017,4,11,201745,201711,20174,Wk-2017/45,Mth-2017/11,Qtr-2017/4,YTD-2017
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,2017,4,11,201745,201711,20174,Wk-2017/45,Mth-2017/11,Qtr-2017/4,YTD-2017
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,2017,2,6,201724,20176,20172,Wk-2017/24,Mth-2017/6,Qtr-2017/2,YTD-2017
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,2016,4,10,201641,201610,20164,Wk-2016/41,Mth-2016/10,Qtr-2016/4,YTD-2016
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,2016,4,10,201641,201610,20164,Wk-2016/41,Mth-2016/10,Qtr-2016/4,YTD-2016


### Convert dates into date datetime datatype  from string datatype
*Coverstion is required as we noticed in above step (df.dtypes)  that date columns are in string/object format , which will act as blocker for applying date functions on data set.*

In [381]:
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

In [382]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [383]:
df[['Order Date','Ship Date']].dtypes 

Order Date    datetime64[ns]
Ship Date     datetime64[ns]
dtype: object

In [449]:
df['Order Date'].max() # Check max date in data set

Timestamp('2018-12-30 00:00:00')

In [451]:
df['Order Date'].min() # Check min date in data set

Timestamp('2015-01-03 00:00:00')

### Create date attrributes from date 
*Below operations are performed to meet standard wbr format reports i.e in week , Month , Quarter and at YTD basis*

In [433]:
df['Order Week']=df['Order Date'].dt.week #getting weeks from Order Date and adding it as a column in data frame.

In [434]:
df['Order Year']=df['Order Date'].dt.year #getting year from Order Date and adding it as a column in data frame.

In [435]:
df['Order quarter']=df['Order Date'].dt.quarter #getting quarter from Order Date and adding it as a column in data frame.

In [436]:
df['Order Month']=df['Order Date'].dt.month #getting month from Order Date and adding it as a column in data frame.

### Creating combined fields
* WBR Dashboard in general contains view for last n weeks , n months , n quarters and YTD to get combined fields created will act as a base for applying rank function.
* Also to avoid addition of two fields (Year and Week No.) during concatination it is converted into string and then moved back to int format.

In [437]:
df['year_week'] = (df['Order Year'].astype(str)+df['Order Week'].astype(str)).astype(int)

In [438]:
df['year_month'] = (df['Order Year'].astype(str)+df['Order Month'].astype(str)).astype(int)

In [439]:
df['year_quarter'] = (df['Order Year'].astype(str)+df['Order quarter'].astype(str)).astype(int)

### Create Column headers for the view
*Below operations are done for creating column names in final dashboard.*

In [440]:
df['Order Week Name']= 'Wk-'+ df['Order Date'].dt.year.astype(str) + '/' + df['Order Date'].dt.week.astype(str)

In [441]:
df['Order Month Name']='Mth-'+df['Order Date'].dt.year.astype(str) + '/' + df['Order Date'].dt.month.astype(str)

In [442]:
df['Order quarter Name']='Qtr-'+df['Order Date'].dt.year.astype(str) + '/' + df['Order Date'].dt.quarter.astype(str)

In [443]:
df['Order year Name']='YTD-'+df['Order Date'].dt.year.astype(str)

In [397]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Order Year,Order quarter,Order Month,year_week,year_month,year_quarter,Order Week Name,Order Month Name,Order quarter Name,Order year Name
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,2017,4,11,201745,201711,20174,Wk-2017/45,Mth-2017/11,Qtr-2017/4,YTD-2017
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,2017,4,11,201745,201711,20174,Wk-2017/45,Mth-2017/11,Qtr-2017/4,YTD-2017
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,2017,2,6,201724,20176,20172,Wk-2017/24,Mth-2017/6,Qtr-2017/2,YTD-2017
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,2016,4,10,201641,201610,20164,Wk-2016/41,Mth-2016/10,Qtr-2016/4,YTD-2016
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,2016,4,10,201641,201610,20164,Wk-2016/41,Mth-2016/10,Qtr-2016/4,YTD-2016


### Create seperate data frame for each timframe  
*Now that we have all attributes required for date field , next step is to create independt dataframes for each time frame.Nevertheless this could have been done on actual dataset but creating independt data frames will help in avoid complications  on group by and each data frame can be checked seperately incase of any error.*

In [398]:
wk_category = pd.DataFrame(df.groupby(['Order Week Name','year_week','Category','Sub-Category']).Sales.sum()).sort_values('year_week' ,ascending=False )

In [399]:
mth_category = pd.DataFrame(df.groupby(['Order Month Name','year_month','Category','Sub-Category']).Sales.sum()).sort_values('year_month' ,ascending=False )

In [400]:
qtr_category = pd.DataFrame(df.groupby(['Order quarter Name','year_quarter','Category','Sub-Category']).Sales.sum()).sort_values('year_quarter' ,ascending=False )

In [403]:
ytd_category = pd.DataFrame(df.groupby(['Order year Name','Order Year','Category','Sub-Category']).Sales.sum()).sort_values('Order Year' ,ascending=False )


### Resetting the Index

In [404]:
wk_category.reset_index(inplace = True)

In [405]:
mth_category.reset_index(inplace = True)

In [406]:
qtr_category.reset_index(inplace = True)

In [407]:
ytd_category.reset_index(inplace = True)

In [408]:
ytd_category.head()

,Order year Name,Order Year,Category,Sub-Category,Sales
0,YTD-2018,2018,Technology,Phones,105340.5160
1,YTD-2018,2018,Office Supplies,Fasteners,857.5940
2,YTD-2018,2018,Furniture,Bookcases,30024.2797
3,YTD-2018,2018,Furniture,Furnishings,28915.0940
4,YTD-2018,2018,Furniture,Tables,60893.5425


### Create Rolling Fields
* Rolling  are created based on combined fileds which are created above for each time frame.
* Rolling fields are created for filtering the data based on x range for each time frame.


In [444]:
wk_category['rolling_week']=wk_category['year_week'].rank(method='dense',ascending=False).astype(int)

In [445]:
mth_category['rolling_month']=mth_category['year_month'].rank(method='dense',ascending=False).astype(int)

In [446]:
qtr_category['rolling_quarter']=qtr_category['year_quarter'].rank(method='dense',ascending=False).astype(int)

In [447]:
ytd_category['rolling_year']=ytd_category['Order Year'].rank(method='dense',ascending=False).astype(int)

### Filter Data based on Rolling Fields

In [413]:
wk_deck = wk_category[wk_category['rolling_week'] <= 6]

In [414]:
mth_deck = mth_category[mth_category['rolling_month'] <= 6]

In [415]:
qtr_deck = qtr_category[qtr_category['rolling_quarter'] <= 4]

In [416]:
ytd_deck=ytd_category[ytd_category['rolling_year'] == 1]

### Pivoting DataFrame
* Pivot is done on each dataframe created based on date attributes to get view in statndard WBR form i.e date range  on top and dimensions on left.
* For below operations in index  two columns are taken (Category & Sub-Category) as index to get data at that granuality.

In [417]:
week_data=wk_deck.pivot_table(index=['Category','Sub-Category'], columns='Order Week Name', values='Sales')

In [418]:
month_data=mth_deck.pivot_table(index=['Category','Sub-Category'], columns='Order Month Name', values='Sales')

In [419]:
quarter_data=qtr_deck.pivot_table(index=['Category','Sub-Category'], columns='Order quarter Name', values='Sales')

In [420]:
ytd_data=ytd_deck.pivot_table(index=['Category','Sub-Category'], columns='Order year Name', values='Sales')

### Adding all data frames as list

In [421]:
dashboard = [week_data, month_data, quarter_data , ytd_data]

### Joining all DataFrames
*All data frames are joined(outer) together to create a one single view.*

In [422]:
final_dashboard = reduce(lambda  left,right: pd.merge(left,right,on=['Category','Sub-Category'],
                                            how='outer'), dashboard)

In [423]:
final_dashboard

Wk-2018/47  Wk-2018/48  Wk-2018/49  Wk-2018/50  \
Category        Sub-Category                                                   
Furniture       Bookcases       1588.955     384.312    1945.006     287.976   
                Chairs          3756.098    7922.060    2115.724     827.686   
                Furnishings     1146.310    4380.618     800.938    1104.116   
                Tables          1651.098    4104.942    2422.312    3893.045   
Office Supplies Appliances      2594.962    1723.760    2245.326     505.252   
                Art              434.902     523.424      97.268     167.198   
                Binders         2665.966    1852.308    2652.507    1128.449   
                Envelopes        249.548     184.640      57.576      61.840   
                Fasteners         44.904      54.658       3.552      23.820   
                Labels            11.520      78.586     118.840      53.040   
                Paper            601.698     778.862    1469.040     139.666   
                Storage         2806.278    2176.850    1121.986     589.090   
                Supplies          11.032       4.360      47.320      25.710   
Technology      Accessories      300.662     848.252    2119.592    1631.232   
                Copiers              NaN         NaN    2599.956    1199.980   
                Machines        1039.992         NaN     649.000         NaN   
                Phones          3532.424   10981.268    1071.298     703.664   

                              Wk-2018/51  Wk-2018/52  Mth-2017/10  \
Category        Sub-Category                                        
Furniture       Bookcases        442.831    593.9728     1376.448   
                Chairs          3375.488   4266.6840     5910.879   
                Furnishings      554.664    297.3420     2484.482   
                Tables          2136.826    273.0600     2100.768   
Office Supplies Appliances      2828.938    245.2100     2956.470   
                Art              151.338    109.2200      454.042   
                Binders          245.471    575.6030     6694.876   
                Envelopes        190.764         NaN      176.948   
                Fasteners          4.824     85.5460      183.968   
                Labels            18.450     72.4500      145.752   
                Paper           1016.254   1019.3820     1501.190   
                Storage         3545.948   3294.0540     4039.972   
                Supplies        2444.260     44.6880      128.576   
Technology      Accessories     1027.074    190.7280     3503.112   
                Copiers              NaN   2879.9520    22899.860   
                Machines             NaN    599.9700     1541.310   
                Phones          1473.938    663.0240     3589.092   

                              Mth-2017/11  Mth-2017/12  Mth-2018/10  \
Category        Sub-Category                                          
Furniture       Bookcases       5797.2748     1616.702    4079.7262   
                Chairs         13173.0830    14950.632    4669.4340   
                Furnishings     4034.7780     5548.522    1724.1320   
                Tables          8778.4930    14562.859   11410.7760   
Office Supplies Appliances      2978.0640     4752.051    3335.5280   
                Art              926.7220      396.282     642.2380   
                Binders         4929.3980    17225.247   10139.3180   
                Envelopes        807.5600      659.932     429.8520   
                Fasteners        192.4080       96.314      72.3560   
                Labels           910.6920      239.792     392.8500   
                Paper           2328.7760     2326.810    1846.6300   
                Storage         7057.6320    10590.618    6104.2880   
                Supplies         356.0260     1710.520      74.1320   
Technology      Accessories     8081.3760     5752.470    5032.0620   
                Copiers         1199.9760     2479.944   1119

###### Testing the output

In [424]:
final_dashboard.shape

(17, 17)

In [425]:
week_data.shape

(17, 6)

In [426]:
month_data.shape

(17, 6)

In [427]:
quarter_data.shape

(17, 4)

In [428]:
ytd_data.shape

(17, 1)